In [6]:
# Mirai_gen_train.tsv, Mirai_gen_validation.tsv, Mirai_gen_test.tsv merged into Mirai_gen_mal.tsv
import csv
f = open('Mirai_gen_mal.tsv','w')
writer = csv.writer(f, delimiter='\t')
with open('Mirai_gen_train.tsv', 'r', encoding="utf8") as x,open('Mirai_gen_validation.tsv', 'r', encoding="utf8") as y,open('Mirai_gen_test.tsv', 'r', encoding="utf8") as z:
    x_reader = csv.reader(x, delimiter='\t')
    y_reader = csv.reader(y, delimiter='\t')
    z_reader = csv.reader(z, delimiter='\t')
    for x_row in x_reader:
        writer.writerow(x_row)
    for y_row in y_reader:
        writer.writerow(y_row)
    for z_row in z_reader:
        writer.writerow(z_row)
f.close()

In [7]:
IEI = 8
E = 1 / IEI 
packets_tsv = '100000_packets_mirai_time_{0}*{1}.tsv'.format(E,IEI)

In [9]:
import csv
# Extract the timestamp of the 100000 expanded packets
list_all_t = []
with open(packets_tsv, 'r', encoding="utf8") as tsv_in:
    tsv_reader = csv.reader(tsv_in, delimiter='\t')
    for row in tsv_reader:
        list_all_t.append(row)
                
import random
import torch
torch.manual_seed(125) 

# First modify the timestamps of all adversarial samples
list_m = []
with open('Mirai_gen_mal.tsv', 'r', encoding="utf8") as tsv_in: 
    tsv_reader = csv.reader(tsv_in, delimiter='\t')
    for row in tsv_reader:
        list_m.append(row)

#Randomly insert fake packets (note: insert the training set and test set separately to ensure that the sample size of the test set accounts for 20%)
# end = float(list_all_t[-1][0])
# start = float(list_all_t[-1 * int(E*IEI) * (30001) - 1][0]) 
# for i in range(0, len(list_m)): 
#     list_m[i][0] = random.uniform(start,end)
start_train = float(list_all_t[-1 * int(E*IEI) * 30001][0]) 
end_train = float(list_all_t[-1 * int(E*IEI) * (6001) - 1][0]) 
for i in range(6001, len(list_m)): 
    list_m[i][0] = random.uniform(start_train,end_train)
start_test = float(list_all_t[-1 * int(E*IEI) * 6001][0]) 
end_test = float(list_all_t[-1][0])
for i in range(0, 6001): 
    list_m[i][0] = random.uniform(start_test,end_test)

# Get the first element of the list
def takeFirst(elem):
    return elem[0]
list_m.sort(key=takeFirst)
    
f = open('Mirai_gen_mal_modify_time_epoch.tsv','w') 
writer = csv.writer(f, delimiter='\t')
for m_row in list_m:
    writer.writerow(m_row)
f.close()

######################################################################################  
# Then insert all adversarial samples with modified timestamps into the original 100,000 data packets
# At the same time, insert the labels of all adversarial samples into the original 100,000 packet label files
list_x = []
list_y = []
list_z = []
list_k = []
# Can't write a line, add ‘\’ to wrap
with open('Mirai_gen_mal_modify_time_epoch.tsv', 'r', encoding="utf8") as x,\
open(packets_tsv, 'r', encoding="utf8") as y,\
open('100000_labels_mirai.tsv', 'r', encoding="utf8") as z: 
    x_reader = csv.reader(x, delimiter='\t')
    y_reader = csv.reader(y, delimiter='\t')
    z_reader = csv.reader(z, delimiter='\t')
    for x_row in x_reader:
        list_x.append(x_row)
        list_k.append('2')
    for y_row in y_reader:
        list_y.append(y_row)
    for z_row in z_reader:
        list_z.append(z_row) 
gen_samples_num = len(list_x)
print(len(list_x))
print(len(list_y))
print(len(list_z))
list_p = []
list_l = []
list_p.append(list_y[0]) # The first line is the field name
del list_y[0]
while len(list_x) > 0 and len(list_y) > 0:
    if list_x[0][0] < list_y[0][0]:
        list_p.append(list_x[0])
        del list_x[0]
        list_l.append(list_k[0])
        del list_k[0]
    else:
        list_p.append(list_y[0])
        del list_y[0]
        list_l.append(list_z[0])
        del list_z[0]
list_p.extend(list_x)
list_p.extend(list_y)
list_l.extend(list_k)
list_l.extend(list_z)
print("merge success")
print(len(list_p))
print(len(list_l))
    
f = open('130001_packets_mirai(insert_all_FP).tsv','w')
writer = csv.writer(f, delimiter='\t')
fla = open('130001_labels_mirai(insert_all_FP).tsv','w')
writerla = csv.writer(fla, delimiter='\t')
for p_row in list_p:
    writer.writerow(p_row)
for l_row in list_l:
    writerla.writerow(l_row)
f.close()
fla.close()

30001
100001
100000
merge success
130002
130001


In [10]:
def encode(s):
    return ' '.join([bin(ord(c)).replace('0b', '') for c in s])

def find_all(sub,s):
    index_list = []
    index = s.find(sub)
    while index != -1:
        index_list.append(index)
        index = s.find(sub,index+1)
        
    if len(index_list) > 0:
        return index_list
    else:
        return -1

def get_next_vector():
    row = tsv_reader.__next__()
    #print(row)
    rowStr = ','.join(row)
    #print(rowStr)
    rowStrEnc = encode(rowStr)
#     print(rowStrEnc)
#     print()
    index_list = find_all(' 101100',rowStrEnc) #find ','
    mask = '0'
    mask = mask.ljust(len(rowStrEnc),'0')
    mask = list(mask)
    modify_idx = [2,3,10,11,12,13,14,15,16,17,18] 
    #modify_idx = [2,3,4,5,14,16,17,18] 
    for i in modify_idx:
        if i == 0:
            head = 0
        else:
            head = index_list[i-1]+len(' 101100')
        if i == 18:
            tail = len(rowStrEnc)
        else:
            tail = index_list[i]
        for j in range(head,tail):
            mask[j]='1'
    index_list = find_all(' 111010',rowStrEnc) #find ':'
    for i in index_list:
        head = i
        tail = head+len(' 111010')
        for j in range(head,tail):
            mask[j]='0'
    index_list = find_all(' 101110',rowStrEnc) #find '.'
    for i in index_list:
        head = i
        tail = head+len(' 101110')
        for j in range(head,tail):
            mask[j]='0'
    mask = ''.join(mask)
#     print(mask)
#     print()
    row_enc_mask = rowStrEnc
    row_enc_mask = list(row_enc_mask)
    for i in range(len(rowStrEnc)):
        if rowStrEnc[i]==' ':
            row_enc_mask[i]='0'
        elif mask[i]=='0':
            row_enc_mask[i]='0'
    row_enc_mask=''.join(row_enc_mask)
#     print(row_enc_mask)
#     print()
    return row_enc_mask

f = open('Mirai_vector_130001.txt','w')
with open('130001_packets_mirai(insert_all_FP).tsv', 'r', encoding="utf8") as tsv_in:
    tsv_reader = csv.reader(tsv_in, delimiter='\t')
    row = tsv_reader.__next__()
    print(row)
    for i in range(0,130001): 
        vector = get_next_vector()
        f.write(vector+'\n')
f.close()

#Find vector maximum
max_vector_len = 0
for line in open("Mirai_vector_130001.txt"):
    max_vector_len = max(max_vector_len,len(line)-1) #Not including'\n'
print("max_vector_len={0}".format(max_vector_len))

#The length is unified to the maximum value, and mal and ben are stored separately
f1 = open('Mirai_vector_mix_60002.txt','w')
f2 = open('Mirai_vector_ben_69999.txt','w')
with open("Mirai_vector_130001.txt",'r') as y:
    line2=y.readlines()
    for i in range(0, 130001): 
        ss=line2[i]
        ss=ss.strip('\n')
        ss=ss.ljust(max_vector_len,'0')
        if i < 69999: 
            f2.write(ss+'\n')
        else:
            f1.write(ss+'\n')
f1.close()
f2.close()

['frame.time_epoch', 'frame.len', 'eth.src', 'eth.dst', 'ip.src', 'ip.dst', 'tcp.srcport', 'tcp.dstport', 'udp.srcport', 'udp.dstport', 'icmp.type', 'icmp.code', 'arp.opcode', 'arp.src.hw_mac', 'arp.src.proto_ipv4', 'arp.dst.hw_mac', 'arp.dst.proto_ipv4', 'ipv6.src', 'ipv6.dst']
max_vector_len=1350


In [18]:
import os
import subprocess
import csv
import numpy as np
import math
import torch
import torch.nn as nn
import torch.optim as optim
from torch.autograd import Variable

torch.manual_seed(50)

def extract(v):
    return v.data.storage().tolist()

def init_weights(m):
    if type(m) == nn.Linear:
        nn.init.xavier_uniform_(m.weight.data, gain=1)
        
class Generator(nn.Module): 
    def __init__(self, input_size, hidden_size, output_size, f): 
        super(Generator, self).__init__()
        self.map1 = nn.Linear(input_size, hidden_size)
        self.map4 = nn.Linear(hidden_size, output_size)
        self.f = f 
 
    def forward(self, x):
        x = self.f(self.map1(x))
        return self.f(self.map4(x))

class Discriminator(nn.Module): 
    def __init__(self, input_size, hidden_size, output_size, f):
        super(Discriminator, self).__init__()
        self.map1 = nn.Linear(input_size, hidden_size)
        self.map3 = nn.Linear(hidden_size, output_size)
        self.f = f 

    def forward(self, x):
        x = self.f(self.map1(x))
        return self.map3(x)

def load_data(filename):
    x = []
    with open(filename,'r') as data:
        lines = data.readlines()
        for line in lines:
            line=line.strip('\n')
            x.append(list(line))
    x = np.array(x,dtype=np.float64)
    return x

# Model parameters
feature_dims = max_vector_len
z_dims = 20
g_input_size = feature_dims + z_dims
g_output_size = feature_dims
g_hidden_size = int(math.sqrt(g_input_size+g_output_size)+10)
d_input_size = g_output_size
d_output_size = 1
d_hidden_size = int(math.sqrt(d_input_size+d_output_size)+10)

# 150*32*5=24000<=30001*0.8=24000
epochs = 150 
batch_size = 32
print_interval = 10

d_learning_rate = 5e-5
g_learning_rate = 5e-5

clip = [-0.01, 0.01] 

d_steps = 5 # 'k' steps in the original GAN paper. Can put the discriminator on higher training freq than generator
g_steps = 1

discriminator_activation_function = torch.sigmoid
generator_activation_function = torch.sigmoid

benign = torch.zeros([batch_size,1]).reshape(-1)
mal = torch.ones([batch_size,1]).reshape(-1)

G = Generator(input_size=g_input_size,
              hidden_size=g_hidden_size,
              output_size=g_output_size,
              f=generator_activation_function)
G.apply(init_weights)
D = Discriminator(input_size=d_input_size,
                  hidden_size=d_hidden_size,
                  output_size=d_output_size,
                  f=discriminator_activation_function)  
D.apply(init_weights)
d_optimizer = optim.RMSprop(D.parameters(), lr=d_learning_rate)
g_optimizer = optim.RMSprop(G.parameters(), lr=g_learning_rate)

# Load the dataset
xmal = load_data('Mirai_vector_mix_60002.txt')
ymal = []
with open("130001_labels_mirai(insert_all_FP).tsv",'r') as x:
    line1=x.readlines()
    for i in range(69999, 130001): 
        if line1[i].find('1') != -1:
            ymal.append(1)
        elif line1[i].find('2') != -1:
            ymal.append(2)
ymal = np.array(ymal)[:, np.newaxis] 
xben = load_data('Mirai_vector_ben_69999.txt')
yben = np.zeros((xben.shape[0], 1), dtype=np.int)
print("len of yben: %d" % (yben.shape[0]))

# Malicious packets are divided into training set and test set
train_size = int(30001 * 0.8) 
cnt = 0
mix_train_size = 0
xmal_list = xmal.tolist()
xtrain_mal = []
with open("130001_labels_mirai(insert_all_FP).tsv",'r') as x:
    line1=x.readlines()
    for i in range(69999, 130001): 
        mix_train_size += 1
        if line1[i].find('1') != -1:
            cnt += 1
            xtrain_mal.append(xmal_list[i - 69999]) 
        if cnt == train_size:
            break
xtrain_mal = np.array(xtrain_mal)
print("train_size: %d" % (train_size))
print("mix_train_size: %d" % (mix_train_size))
xtest_mal = xmal[mix_train_size:, :]
ytrain_mal = np.ones((train_size, 1), dtype=np.int)
ytest_mal = ymal[mix_train_size:, :]
print("len of xtrain_mal: %d" % (xtrain_mal.shape[0]))
print("len of xtest_mal: %d" % (xtest_mal.shape[0]))
mal_tot = 0
dis_tot = 0
for array in ytest_mal:
    if array[0] == 1:
        mal_tot += 1
    elif array[0] == 2:
        dis_tot += 1
print("len of mal: %d" % (mal_tot))
print("len of dis: %d" % (dis_tot))
# The benign packets are divided into training set and test set, the number of training sets is the same as the training set of malicious packets
xtrain_ben = xben[0:train_size, :]
xtest_ben = xben[train_size:, :]
ytrain_ben = yben[0:train_size, :]
ytest_ben = yben[train_size:, :]

xtrain_mal = torch.Tensor(xtrain_mal)
xtest_mal = torch.Tensor(xtest_mal)
xtrain_ben = torch.Tensor(xtrain_ben)
xtest_ben = torch.Tensor(xtest_ben)

index_of_data_D_ben = 0     
index_of_data_D_mal = 0    
index_of_data_G = 0

for epoch in range(1, epochs+1):
    for d_index in range(d_steps):
        D.zero_grad()
        
        #  1A: Train D on real
        idx = np.array(range(index_of_data_D_ben, index_of_data_D_ben+batch_size))
        index_of_data_D_ben += batch_size
        xben_batch = xtrain_ben[idx]
        d_real_data = Variable(xben_batch)
        d_real_decision = D(d_real_data)
        d_real_error = torch.mean(d_real_decision.reshape(-1))
        d_real_error.backward(benign.mean())
        
        #  1B: Train D on fake
        idx = np.array(range(index_of_data_D_mal, index_of_data_D_mal+batch_size))
        index_of_data_D_mal += batch_size
        xmal_batch = xtrain_mal[idx]
        np.random.seed(0)
        noise = np.random.uniform(0, 1, (batch_size, z_dims))
        d_gen_input = Variable(torch.cat((xmal_batch, torch.Tensor(noise)), 1))
        d_fake_data = G(d_gen_input).detach() # detach to avoid training G on these labels
        d_fake_decision = D(d_fake_data)
        d_fake_error = torch.mean(d_fake_decision.reshape(-1))
        d_fake_error.backward(mal.mean())
        
        d_optimizer.step() # Only optimizes D's parameters; changes based on stored gradients from backward()
        
        # Clip weights of discriminator
        for p in D.parameters():
            p.data.clamp_(clip[0], clip[1])
            
        dre, dfe = extract(d_real_error)[0], extract(d_fake_error)[0]
    
    for g_index in range(g_steps):
        G.zero_grad()
        
        idx = np.array(range(index_of_data_G, index_of_data_G+batch_size))
        index_of_data_G += batch_size
        xmal_batch = xtrain_mal[idx]
        np.random.seed(0)
        noise = np.random.uniform(0, 1, (batch_size, z_dims))
        gen_input = Variable(torch.cat((xmal_batch, torch.Tensor(noise)), 1))
        gen_examples = G(gen_input)
        dg_fake_decision = D(gen_examples)
        g_error = torch.mean(dg_fake_decision.reshape(-1))
        g_error.backward(benign.mean())
        g_loss = extract(g_error)[0]
        
        g_optimizer.step()  # Only optimizes G's parameters

    if epoch % print_interval == 0:
        # D(benign)
        d_real_data = Variable(xtest_ben)
        d_real_decision = D(d_real_data)
        d_real_error = torch.mean(d_real_decision.reshape(-1))
        d_loss = extract(d_real_error)[0]
        #print("len of d_loss: %d" % (len(d_loss)))
        print("d_loss: %f" % (d_loss))
        # D(G(benign))
        np.random.seed(0)
        noise = np.random.uniform(0, 1, (xben.shape[0] - train_size, z_dims))
        gen_input = Variable(torch.cat((xtest_ben, torch.Tensor(noise)), 1))
        gen_examples = G(gen_input)
        dg_fake_decision = D(Variable(gen_examples))
        dg_error = torch.mean(dg_fake_decision.reshape(-1))
        dg_loss = extract(dg_error)[0]
        #print("len of dg_loss: %f" % (len(dg_loss)))
        print("dg_loss: %f" % (dg_loss))

#         threshold = 0
#         for i in range(0, len(d_loss)):
#             threshold = max(threshold, abs(d_loss[i]-dg_loss[i]))
        threshold = abs(d_loss-dg_loss)
        print("threshold: %f" % (threshold)) 
        
        # D(G(Malicious))
        np.random.seed(0)
        noise = np.random.uniform(0, 1, (xmal.shape[0] - mix_train_size, z_dims))
        gen_input = Variable(torch.cat((xtest_mal, torch.Tensor(noise)), 1))
        gen_examples = G(gen_input)
        dg_fake_decision = D(Variable(gen_examples))
        dg_error = dg_fake_decision.reshape(-1)
        dg_loss = extract(dg_error)
        print("len of dg_loss: %d" % (len(dg_loss)))
        
        # Calculation Precision Recall F-score MRR
        positive_sum = 6001
        positive_num = 0
        FP = 0
        label_list = []
        for i in range(0, len(dg_loss)):
            if abs(d_loss-dg_loss[i]) > threshold:
                label_list.append('1')
                if ytest_mal[i][0] == 0 or ytest_mal[i][0] == 2:
                    FP += 1
                elif ytest_mal[i][0] == 1:
                    positive_num += 1
            else:
                label_list.append('0')
        

        label_str = ''.join(label_list)
        file = open('6001_mirai_labels_gan.txt','a')
        file.write(label_str + '\n')
        file.close()
        
        if positive_num == 0:
            Precision = 0
            Recall = 0
            F_score = 0
        else:
            Precision = positive_num / (positive_num + FP)
            Recall = positive_num / positive_sum
            F_score = 2 * Recall * Precision / (Recall + Precision)
        origin_mal_num = 5510 
        MRR = (origin_mal_num - positive_num) / origin_mal_num
        print('Precision: {0:0.4f}'.format(Precision)) 
        print('Recall: {0:0.4f}'.format(Recall)) 
        print('F-score: {0:0.4f}'.format(F_score)) 
        print('MRR: {0} - {1} / {2} = {3:0.4f}'.format(origin_mal_num,positive_num,origin_mal_num,MRR))
        print("%d [D real_err: %f, fake_err: %f] [G loss: %f]" % (epoch, dre, dfe, g_loss)) 


len of yben: 69999
train_size: 24000
mix_train_size: 48000
len of xtrain_mal: 24000
len of xtest_mal: 12002
len of mal: 6001
len of dis: 6001
d_loss: -0.092868
dg_loss: -0.148406
threshold: 0.055538
len of dg_loss: 12002
Precision: 0.2854
Recall: 0.2468
F-score: 0.2647
MRR: 5510 - 1481 / 5510 = 0.7312
10 [D real_err: -0.092279, fake_err: -0.146997] [G loss: -0.148408]
d_loss: -0.130221
dg_loss: -0.198314
threshold: 0.068093
len of dg_loss: 12002
Precision: 0.3018
Recall: 0.2693
F-score: 0.2846
MRR: 5510 - 1616 / 5510 = 0.7067
20 [D real_err: -0.128558, fake_err: -0.197547] [G loss: -0.198325]
d_loss: -0.161208
dg_loss: -0.231878
threshold: 0.070670
len of dg_loss: 12002
Precision: 0.3191
Recall: 0.3096
F-score: 0.3143
MRR: 5510 - 1858 / 5510 = 0.6628
30 [D real_err: -0.160437, fake_err: -0.231222] [G loss: -0.231885]
d_loss: -0.196145
dg_loss: -0.274450
threshold: 0.078305
len of dg_loss: 12002
Precision: 0.5556
Recall: 0.8910
F-score: 0.6844
MRR: 5510 - 5347 / 5510 = 0.0296
40 [D real

In [ ]:
#
torch.manual_seed(125)
d_loss: -0.209761
dg_loss: -0.296548
threshold: 0.086788
len of dg_loss: 12002
Precision: 0.6444
Recall: 0.9723
F-score: 0.7751
MRR: 5510 - 5835 / 5510 = -0.0590
40 [D real_err: -0.208606, fake_err: -0.295453] [G loss: -0.296602]
#
torch.manual_seed(0)
d_loss: -0.315643
dg_loss: -0.465664
threshold: 0.150022
len of dg_loss: 12002
Precision: 0.5888
Recall: 0.8775
F-score: 0.7047
MRR: 5510 - 5266 / 5510 = 0.0443
130 [D real_err: -0.316925, fake_err: -0.465663] [G loss: -0.465679]
# *
torch.manual_seed(200)
d_loss: -0.321863
dg_loss: -0.467227
threshold: 0.145364
len of dg_loss: 12002
Precision: 0.5418
Recall: 0.9733
F-score: 0.6961
MRR: 5510 - 5841 / 5510 = -0.0601
150 [D real_err: -0.318696, fake_err: -0.467217] [G loss: -0.467229]
#
torch.manual_seed(100)
d_loss: -0.240253
dg_loss: -0.349383
threshold: 0.109130
len of dg_loss: 12002
Precision: 0.6299
Recall: 0.9265
F-score: 0.7499
MRR: 5510 - 5560 / 5510 = -0.0091
50 [D real_err: -0.239902, fake_err: -0.347397] [G loss: -0.349398]
#
torch.manual_seed(50)
d_loss: -0.245166
dg_loss: -0.357638
threshold: 0.112472
len of dg_loss: 12002
Precision: 0.5631
Recall: 0.9607
F-score: 0.7100
MRR: 5510 - 5765 / 5510 = -0.0463
50 [D real_err: -0.244772, fake_err: -0.355752] [G loss: -0.357657]

In [ ]:
# Train the fake packets as malicious packets, the result is unstable, hereby record (no random seed is set)
#11111111111111111111111111111111111
d_loss: -0.322740
dg_loss: -0.467157
threshold: 0.144417
len of dg_loss: 10188.000000
mal_TPR: 5788 / 6001 = 0.9645
dis_TPR: 2946 / 4187 = 0.7036
dis_mal_TPR: 8734 / 10188 = 0.8573
MRR: 5510 - 5788 / 5510 = -0.0505
150 [D real_err: -0.318539, fake_err: -0.467148] [G loss: -0.467162]
#22222222222222222222222222222222222
d_loss: -0.322835
dg_loss: -0.467304
threshold: 0.144469
len of dg_loss: 10188.000000
mal_TPR: 282 / 6001 = 0.0470
dis_TPR: 1693 / 4187 = 0.4043
dis_mal_TPR: 1975 / 10188 = 0.1939
MRR: 5510 - 282 / 5510 = 0.9488
150 [D real_err: -0.318586, fake_err: -0.467286] [G loss: -0.467302]
#333333333333333333333333333333333333
d_loss: -0.321234
dg_loss: -0.467102
threshold: 0.145868
len of dg_loss: 10188.000000
mal_TPR: 5663 / 6001 = 0.9437
dis_TPR: 4017 / 4187 = 0.9594
dis_mal_TPR: 9680 / 10188 = 0.9501
MRR: 5510 - 5663 / 5510 = -0.0278
150 [D real_err: -0.316993, fake_err: -0.467095] [G loss: -0.467110]
#44444444444444444444444444444444444
d_loss: -0.324198
dg_loss: -0.467553
threshold: 0.143356
len of dg_loss: 10188.000000
mal_TPR: 202 / 6001 = 0.0337
dis_TPR: 1205 / 4187 = 0.2878
dis_mal_TPR: 1407 / 10188 = 0.1381
MRR: 5510 - 202 / 5510 = 0.9633
150 [D real_err: -0.319922, fake_err: -0.467534] [G loss: -0.467549]
#55555555555555555555555555555555555
d_loss: -0.322354
dg_loss: -0.467157
threshold: 0.144803
len of dg_loss: 10188.000000
mal_TPR: 83 / 6001 = 0.0138
dis_TPR: 1203 / 4187 = 0.2873
dis_mal_TPR: 1286 / 10188 = 0.1262
MRR: 5510 - 83 / 5510 = 0.9849
150 [D real_err: -0.318029, fake_err: -0.467139] [G loss: -0.467153]
#66666666666666666666666666666666666
d_loss: -0.322957
dg_loss: -0.467438
threshold: 0.144481
len of dg_loss: 10188.000000
mal_TPR: 4360 / 6001 = 0.7265
dis_TPR: 1770 / 4187 = 0.4227
dis_mal_TPR: 6130 / 10188 = 0.6017
MRR: 5510 - 4360 / 5510 = 0.2087
150 [D real_err: -0.318654, fake_err: -0.467423] [G loss: -0.467437]
#77777777777777777777777777777777777
d_loss: -0.322979
dg_loss: -0.467327
threshold: 0.144348
len of dg_loss: 10188.000000
mal_TPR: 4930 / 6001 = 0.8215
dis_TPR: 992 / 4187 = 0.2369
dis_mal_TPR: 5922 / 10188 = 0.5813
MRR: 5510 - 4930 / 5510 = 0.1053
150 [D real_err: -0.318737, fake_err: -0.467315] [G loss: -0.467326]
#88888888888888888888888888888888888
d_loss: -0.322789
dg_loss: -0.467399
threshold: 0.144610
len of dg_loss: 10188.000000
mal_TPR: 49 / 6001 = 0.0082
dis_TPR: 64 / 4187 = 0.0153
dis_mal_TPR: 113 / 10188 = 0.0111
MRR: 5510 - 49 / 5510 = 0.9911
150 [D real_err: -0.318498, fake_err: -0.467379] [G loss: -0.467392]
#99999999999999999999999999999999999
d_loss: -0.323224
dg_loss: -0.467356
threshold: 0.144132
len of dg_loss: 10188.000000
mal_TPR: 1548 / 6001 = 0.2580
dis_TPR: 292 / 4187 = 0.0697
dis_mal_TPR: 1840 / 10188 = 0.1806
MRR: 5510 - 1548 / 5510 = 0.7191
150 [D real_err: -0.318809, fake_err: -0.467338] [G loss: -0.467349]
#00000000000000000000000000000000000
d_loss: -0.323670
dg_loss: -0.467358
threshold: 0.143687
len of dg_loss: 10188.000000
mal_TPR: 5727 / 6001 = 0.9543
dis_TPR: 1380 / 4187 = 0.3296
dis_mal_TPR: 7107 / 10188 = 0.6976
MRR: 5510 - 5727 / 5510 = -0.0394
150 [D real_err: -0.319449, fake_err: -0.467346] [G loss: -0.467357]